## Analyses Scripts

##### Author: A. Zarkali
##### Date of last update: 14/03/2020
##### Aim: Compare gradients and SCFC across groups

In [2]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import shapiro
from scipy.stats import percentileofscore
import statsmodels.api as sm
#import statsmodels.formula.api as sm
from statsmodels.formula.api import ols
from pathlib import Path
import seaborn as sns
# Enable inline plotting
%matplotlib inline

In [5]:
### Participant Lists
# Load clinical information
clin = pd.read_excel(r"C:\Users\Angelika\Dropbox\PhD\PAPERS\Publications\Structure-Function\NatureComms\SupplementaryData1\Figure2\Data\ClinicalInformation.xlsx")
sfc = pd.read_csv(r"C:\Users\Angelika\Dropbox\PhD\PAPERS\Publications\Structure-Function\NatureComms\SupplementaryData1\Figure2\Data\StructureFuction.csv", index_col=0)
# Select controls only
controls = clin[clin.PD==0].ParticipantNumber
HighVis = clin[(clin.PD == 1) & (clin.PD_VisPerf == 0)].ParticipantNumber
LowVis = clin[(clin.PD == 1) & (clin.PD_VisPerf == 1)].ParticipantNumber
All = clin.ParticipantNumber

In [4]:
### Load files
### SCFC data
df1 = pd.read_csv(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/04_StructureFunction/ANALYSES/StructureFunction/StructureFuction_Spearman_Bin_Exc210.csv", index_col=0)
df1 = df1.transpose() # structure/function and ranks only
df1 = df1.replace(np.nan, 0)
### Clinical data
df2 = pd.read_excel(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/04_StructureFunction/DATA/DataIncluded_Combined_Exc210.xlsx", index_col=0)
### Merge together to signle dataframe
df = pd.concat([df2, df1], axis=1)
### Load subgroup databases
dfPD = df[df.PD == 1] #PD only
dfControl = df[df.PD == 0]

#### Mixed Linear Model across nodes

In [ ]:
# #Mixed linear model
# #which dataframe group to compare
data = dfPD
parameter = "PD_VisPerf"

# Declare dummy group to fit intercept variables
data["group"] = 1  
# Dummy variables
vc = {"Age": "0 +C(Age)", "Gender":"0+C(Gender)"}

# Declare empty lists
pvalues = []
coefficients = []
lowerCI = []
upperCI = []
# loop through all nodes:
for i in range(1,401): # change for cortical gradients to 400
    formula = "Node" + str(i) + " ~ " + parameter 
    md = sm.MixedLM.from_formula(formula, data=data, re_formula="0", vc_formula=vc, groups=data["group"])
    mdf = md.fit()
    p = mdf.pvalues[1]
    coef = (mdf.conf_int().loc[parameter]).mean()
    lower = (mdf.conf_int().loc[parameter])[0]
    upper = (mdf.conf_int().loc[parameter])[1]
    pvalues.append(p)
    coefficients.append(coef)
    lowerCI.append(lower)
    upperCI.append(upper)
#FDR correct
FDR = sm.stats.multipletests(pvalues, is_sorted=False, alpha=0.05, method="fdr_bh", returnsorted=False)

# Merge to Dataframe and export as csv
outdata = pd.DataFrame(data=np.zeros((400,0))) # change for cortical gradients to 400
outdata["Coef"] = coefficients
outdata["lowerCI"] = lowerCI
outdata["upperCI"] = upperCI
outdata["pValues"] = pvalues
outdata["FDR"] = FDR[1]
outname = "SCFC_Schaeffer" + parameter + ".csv"
outdata.to_csv(outname)

#### SC-FC across whole networks

In [ ]:
## Test SC-FC coupling across the whole network
# Add a column of mean SC-FC values across all 400 nodes
data = df
data["MeanSCFC"] = sfc.mean(axis=1)
## ANCOVA with age and gender as covariates
parameter = "C(PD)" ### Parameter to check - change this as needed
data["group"] = 1
vc = {"Age": "0 +C(Age)", "Gender":"0+C(Gender)"}
formula = "MeanSCFC ~ " + parameter + " + Age + C(Gender)"
md = sm.OLS.from_formula(formula=formula, data=data)
mdf = md.fit()
mdf.summary()

#### Spin permutations

In [ ]:
# Load permutations
permutation = pd.read_csv(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/04_StructureFunction/SCRIPTS/permutations_Glasser.txt", header = None, delimiter=" ")
df1 = pd.read_csv(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/04_StructureFunction/DATA/GRADIENTS/Glasser/FunctionalGradientOne_Glasser.csv", index_col=0)
#df1 = pd.read_csv(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/04_StructureFunction/ANALYSES\Gradients\Functional\NA\FunctionalGradient_PD_Grad2_NA.csv", index_col=0')
#corDF = pd.read_csv(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/04_StructureFunction/ANALYSES/StructureFunction/StructureFuction_Spearman_Glasser.csv", index_col=0)
corDF = pd.read_csv(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/04_StructureFunction/ANALYSES/StructureFunction/Glasser/StructureFunction_Glasser_PD.csv", index_col=0)
df1 = df1.transpose()
df1 = df1[PD]
df1 = df1.replace(np.nan,0)
grad = df1.mean(axis=1)
cor = corDF.Coef ### for the change gradients files

n_roi = 360 # Glasser 360, Schaeffer  400
n_perm = 1000
r_spin = np.empty(n_perm)
r_obs, p_obs = stats.spearmanr(cor, grad)

for i in range(n_perm):
    rotated_index = permutation[i]
    perm = np.empty(n_roi)
    for j in range(n_roi):
        perm[j] = cor[(rotated_index[j] - 1)] # -1 as starting from 0
    r_spin[i] = stats.spearmanr(perm, grad)[0]
pv_spin = np.mean(np.abs(r_spin) >= np.abs(r_obs))
r_obs, pv_spin

#### Genetics

In [ ]:
# ### Load genetic data
data = pd.read_csv(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/04_StructureFunction/DATA/Atlas/GeneExpression/schaeffer_expression_mirror.csv")
# Define Transmitters into lists
norepinephrine = ["ADRA1A", "ADRA1B", "ADRA1D", "ADRA2A","ADRA2C"]
acetylcholine = ["CHRM1", "CHRM2", "CHRM3", "CHRM4", "CHRM5", "CHRNA2", "CHRNA3", "CHRNA4", "CHRNA6", "CHRNA7", "CHRNA10", "CHRNB1", "CHRNB2"]
dopamine = ["DRD1", "DRD2", "DRD4"]
serotonin = ["HTR1A", "HTR1E", "HTR1F", "HTR2A", "HTR2C", "HTR3B", "HTR3C", "HTR4", "HTR5A", "HTR7"]
excluded = ['CHRND', 'CHRNA5', 'DRD5', 'ADRA2B', 'CHRNA1', 'CHRNB3', 'CHRNE', 'CHRNA9', 'DRD3', "HTR1B", 'HTR1D', 'HTR2B', 'HTR3A', 'HTR3D', 'HTR3E', 'HTR5B', 'HTR6']
# Select and save transmitters only
allTransmitter = norepinephrine + acetylcholine + dopamine + serotonin
data = data[allTransmitter]
# Concatenate with SCFC and clinical data
df = pd.concat([df,data], axis=1)

In [ ]:
# Concatenate with SCFC and clinical data
df = pd.read_csv(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/04_StructureFunction/ANALYSES/StructureFunction/Glasser/StructureFunction_PD_VisPerf.csv", index_col=0)
data["Coef"] = df.Coef.values
df = pd.concat([df,data], axis=1)

In [ ]:
### Compare SCFC changes with regional gene expression
rho = np.zeros(len(allTransmitter))
pvals = np.zeros(len(allTransmitter))
for i in range(len(allTransmitter)):
    copy = data
    copy = copy[np.isfinite(copy[str(allTransmitter[i])])]
    v1 = copy.Coef
    v2 = copy[allTransmitter[i]]
    r, p = stats.spearmanr(v1,v2)
    rho[i] = r
    pvals[i] = p
FDR = sm.stats.multipletests(pvals, is_sorted=False, alpha=0.05, method="fdr_bh", returnsorted=False)
# Export data
exportData = pd.DataFrame(data=allTransmitter)
exportData["rho"] = rho 
exportData["pValue"] = pvals
exportData["FDR"] = FDR[1]
exportData.to_csv("Transmitter_SF_PD_VisPerf.csv")